In [230]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import pyodbc
import pandas_access as pa
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score,classification_report
from sklearn.model_selection import cross_val_predict,GridSearchCV, StratifiedKFold
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.covariance import EllipticEnvelope
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler  
from sklearn.neural_network import MLPClassifier
from tensorflow import keras
import tensorflow as tf
import imblearn
from imblearn.over_sampling import SMOTE,BorderlineSMOTE,ADASYN,SVMSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as imbpipeline
import seaborn as sns
from collections import Counter
import loras

In [267]:
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)
pd.options.mode.use_inf_as_na = True

In [268]:
dataset=pd.read_csv('data\hospital_closure\hosp_closest_info_v5.csv',thousands=',')

In [190]:
dataset

,Unnamed: 0,hrrnum,year,ssacounty,provider,fips,zip_short.x,prvdr_num,fyb,fybstr,fye,fyestr,status,hospital_name,street_addr,po_box,city,state,zip_code,county,ime1,ime2,ime3,dsh1,dsh2,dsh3,dshpct,gme_part_a,gme_part_b,beds,bed_days,medicare_days,medicaid_days,days,intres,employees,volunteers,medicare_disch,medicaid_disch,discharges,medicaid_hmo_discharges,medicaid_hmo_ipf_subprovider,medicaid_hmo_irf_subprovider,medicaid_ipf_subprovider,medicaid_irf_subprovider,total_discharges_subprovider_ipf,total_discharges_subprovider_irf,overall,doc_communicate,nurse_communicate,help_quick,staff_explain,pain_control,area_quiet,room_clean,discharge_info,recommend,num_complete_survey,survey_response_rate,Hospital.Type,Hospital.Ownership,ehr,rate.x,mortality,safety,readmin,patient_exp,effective,timely,efficient_img,age,sex,race.x,race.y,buyin_mo,type_adm.x,type_adm.y,type_adm,ownership,ind_dsh,ind_teaching,efficiency,utilization,hsanum,prop_discharges,rate.y,rural_urban,pop,unemp,poverty,under_65,prop_phy_pcp,prop_md_pcp,prop_pcp,prop_gi,prop_gen_int,prop_hosp,prop_hosp_short,prop_cah,prop_hosp_adm,prop_hosp_adm_short,prop_hosp_bed,prop_hosp_bed_short,prop_days_inp,prop_days_inp_short,prop_days_inp_med,ind_close,state_cd,id_closed,zip_short.y,year_close_combined,dist,ind_nearest,ind_after_1,ind_after_2,ind_after_3,ind_after_4,ind_after_5,ind_after_6,ind_after_7,ind_after_8,rate,hmo_rate,hhi
0,1,1.0,2001,1130.0,10073,1027.0,36251,10073,10/1/2000,01-OCT-00,9/30/2001,30-SEP-01,Reopened,CLAY COUNTY HOSPITAL,544 EAST AVENUE,36251,ASHLAND,AL,36251,CLAY,NaN,NaN,NaN,NaN,80350.0,NaN,NaN,NaN,NaN,46.0,16790.0,5409.0,NaN,7948,NaN,205.000000,NaN,1082.0,232.0,2100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acute Care Hospitals,Government - Hospital District or Authority,1.0,3.333333,2.000000,NaN,2.333333,2.000000,1.333333,2.000000,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Public,1,0,45.652174,0.473377,1006.0,416.729914,NaN,9.0,13862.0,11.8,20.6,81.791949,0.000361,0.000361,0.000721,0.000000,0.000072,0.000072,0.000072,0.0,0.090463,0.090463,0.009306,0.009306,2.373539,2.373539,0.366542,0,AL,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,NaN,NaN,416.729914
1,2,1.0,2001,1270.0,10046,1055.0,35901,10046,7/1/2001,01-JUL-01,6/30/2002,30-JUN-02,Reopened,RIVERVIEW REG L MEDICAL CENTER,600 SOUTH THIRD STREET,NaN,GADSDEN,AL,35901,ETOWAH,NaN,NaN,NaN,3588783.0,NaN,NaN,NaN,NaN,NaN,281.0,102565.0,36053.0,NaN,54864,NaN,833.900024,NaN,5738.0,1394.0,10424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acute Care Hospitals,Proprietary,1.0,3.000000,2.000000,3.000000,2.000000,1.000000,1.666667,2.333333,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Private,1,0,37.096085,0.534919,1039.0,416.729914,NaN,3.0,104303.0,9.1,19.0,82.708072,0.000719,0.000671,0.002196,0.000048,0.000192,0.000038,0.000019,0.0,0.216120,0.191644,0.005657,0.004631,1.311707,1.027736,0.647872,0,AL,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,NaN,NaN,416.729914
2,3,1.0,2001,1360.0,10011,1073.0,35235,10011,7/1/2001,01-JUL-01,6/30/2002,30-JUN-02,Reopened,MEDICAL CENTER EAST,50 MEDICAL PARK DRIVE EAST,NaN,BIRMINGHAM,AL,35235,NaN,NaN,1456787.0,NaN,892617.0,NaN,NaN,NaN,898821.0,136038.0,237.0,90155.0,26904.0,NaN,59743,NaN,843.000000,NaN,5363.0,903.0,13849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acute Care Hospitals,Voluntary non-profit - Private,1.0,2.333333,2.000000,1.000000,2.333333,1.000000,1.333333,2.000000,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nonprofit,1,1,58.434599,0.690631,1011.0,416.729914,NaN,1.0,658931.0,8.7,18.6,84.804479,0.000982,0.000952,0.006362,0.000091,0.000756,0.000023,0.000014,0.0,0.264985,0.232287,0.006441,0.005428,1.582758,1.302935,0.586647,0,AL,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,NaN,NaN,416.729914
3,4,1.0,2001,1360.0,10104,1073.0,35213,10104,7/1/2001,01-JUL-01,6/30/2002,30-JUN-02,Reopened,BMC - MONTCLAIR,800 MONTCLAIR ROAD,NaN,BIRMINGHAM,AL,35213,JEFFERSON,NaN,3008272.0,NaN,1761034.0,NaN,NaN,NaN,2051016.0,155872.0,341.0,124465.0,41015.0,

In [191]:
def check(a, b):
    if a == b-1:
        return 1
    else:
        return 0

In [288]:
dataset.drop(dataset[dataset['year']==2017].index,inplace=True)
dataset['year_close_combined'].fillna(0,inplace=True)
dataset['Closure']=dataset.apply(lambda x:check(float(x['year']),x['year_close_combined']),axis=1)

In [289]:
dataset_=dataset[['provider','dshpct','gme_part_a','gme_part_b','beds','bed_days','medicare_days','medicaid_days','days','employees','medicare_disch','medicaid_disch','discharges','doc_communicate','nurse_communicate','help_quick','staff_explain','pain_control','area_quiet','room_clean','discharge_info','recommend','num_complete_survey','survey_response_rate','rate.x','mortality','safety','readmin','patient_exp','effective','timely','sex','race.x','race.y','buyin_mo','type_adm.x','type_adm.y','type_adm','ind_dsh','ind_teaching','efficiency','utilization','hsanum','prop_discharges','rate.y','rural_urban','pop','unemp','poverty','under_65','prop_phy_pcp','prop_md_pcp','prop_pcp','prop_gi','prop_gen_int','prop_hosp','prop_hosp_short','prop_cah','prop_hosp_adm','prop_hosp_adm_short','prop_hosp_bed','prop_hosp_bed_short','prop_days_inp','prop_days_inp_short','prop_days_inp_med','Closure']]

In [290]:
dataset_['provider']=pd.to_numeric(dataset_['provider']).astype(float)
dataset_.loc[dataset_['provider']==100024,'Closure']=0
dataset_.loc[dataset_['provider']==40021,'Closure']=0
dataset_.loc[dataset_['provider']==200025,'Closure']=0
dataset_.loc[dataset_['provider']==260147,'Closure']=0
dataset_.loc[dataset_['provider']==440047,'Closure']=0
dataset_.loc[dataset_['provider']==180149,'Closure']=0
dataset_.loc[dataset_['provider']==450832,'Closure']=0
dataset_.loc[dataset_['provider']==670052,'Closure']=0
dataset_.loc[dataset_['provider']==100015,'Closure']=0
dataset_.loc[dataset_['provider']==330225,'Closure']=0
dataset_.loc[dataset_['provider']==360347,'Closure']=0
dataset_.loc[dataset_['provider']==390108,'Closure']=0
dataset_.loc[dataset_['provider']==150170,'Closure']=0
dataset_.loc[dataset_['provider']==40142,'Closure']=0
dataset_.loc[dataset_['provider']==390131,'Closure']=0
dataset_.loc[dataset_['provider']==390010,'Closure']=0
dataset_.loc[dataset_['provider']==330036,'Closure']=0

<ipython-input-290-a20e6259be24>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_['provider']=pd.to_numeric(dataset_['provider']).astype(float)
C:\Users\zys\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [293]:
dataset_close=dataset_[dataset_['Closure']==1]
dataset_open=dataset_[dataset_['Closure']==0]

In [295]:
dataset_close

,provider,dshpct,gme_part_a,gme_part_b,beds,bed_days,medicare_days,medicaid_days,days,employees,medicare_disch,medicaid_disch,discharges,doc_communicate,nurse_communicate,help_quick,staff_explain,pain_control,area_quiet,room_clean,discharge_info,recommend,num_complete_survey,survey_response_rate,rate.x,mortality,safety,readmin,patient_exp,effective,timely,sex,race.x,race.y,buyin_mo,type_adm.x,type_adm.y,type_adm,ind_dsh,ind_teaching,efficiency,utilization,hsanum,prop_discharges,rate.y,rural_urban,pop,unemp,poverty,under_65,prop_phy_pcp,prop_md_pcp,prop_pcp,prop_gi,prop_gen_int,prop_hosp,prop_hosp_short,prop_cah,prop_hosp_adm,prop_hosp_adm_short,prop_hosp_bed,prop_hosp_bed_short,prop_days_inp,prop_days_inp_short,prop_days_inp_med,Closure
137,10056.0,NaN,19973.0,5480.0,325.0,118625.0,34682.0,NaN,86859,1693.219971,6324.0,296.0,20292,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.333333,2.000000,1.000000,3.000000,3.000000,1.333333,1.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,62.436923,0.732215,1011.0,439.583065,NaN,1.0,658931.0,8.7,18.6,84.804479,0.000982,0.000952,0.006362,0.000091,0.000756,0.000023,0.000014,0.000000,0.264985,0.232287,0.006441,0.005428,1.582758,1.302935,0.586647,1
357,10033.0,NaN,7331708.0,782565.0,967.0,352955.0,66264.0,NaN,249945,6106.000000,10620.0,6276.0,38460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.666667,2.333333,2.333333,1.000000,3.000000,1.666667,1.000000,0.468966,0.861084,0.133990,0.111330,0.319212,0.278818,0.400985,1,1,39.772492,0.708150,1011.0,471.861666,757.470001,1.0,658931.0,8.7,18.6,84.804479,0.000982,0.000952,0.006362,0.000091,0.000756,0.000023,0.000014,0.000000,0.264985,0.232287,0.006441,0.005428,1.582758,1.302935,0.586647,1
364,10018.0,NaN,27551.0,256402.0,20.0,7300.0,280.0,NaN,709,145.410004,69.0,29.0,310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,2.000000,0.250000,0.937500,0.062500,0.187500,0.375000,0.187500,0.437500,0,1,15.500000,0.097123,1011.0,471.861666,757.470001,1.0,658931.0,8.7,18.6,84.804479,0.000982,0.000952,0.006362,0.000091,0.000756,0.000023,0.000014,0.000000,0.264985,0.232287,0.006441,0.005428,1.582758,1.302935,0.586647,1
377,10005.0,NaN,NaN,NaN,108.0,39528.0,13260.0,NaN,24955,597.159973,2813.0,1382.0,6904,90,89,73,86,83,77,83,98,90,300 or More,80,2.666667,1.000000,2.000000,3.000000,2.000000,2.000000,3.000000,0.362360,0.994382,0.000000,0.460674,0.716292,0.252809,0.030899,1,0,63.925926,0.633054,1012.0,456.241031,733.020020,4.0,94166.0,8.6,19.4,83.464308,0.000552,0.000520,0.001211,0.000011,0.000085,0.000021,0.000021,0.000000,0.113194,0.113194,0.002145,0.002145,0.420545,0.420545,0.239588,1
387,10005.0,NaN,NaN,NaN,108.0,39528.0,13260.0,NaN,24955,597.159973,2813.0,1382.0,6904,90,89,73,86,83,77,83,98,90,300 or More,80,2.666667,1.000000,2.000000,3.000000,2.000000,2.000000,3.000000,0.362360,0.994382,0.000000,0.460674,0.716292,0.252809,0.030899,1,0,63.925926,0.633054,1012.0,456.241031,733.020020,4.0,94166.0,8.6,19.4,83.464308,0.000552,0.000520,0.001211,0.000011,0.000085,0.000021,0.000021,0.000000,0.113194,0.113194,0.002145,0.002145,0.420545,0.420545,0.239588,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95617,510086.0,NaN,NaN,NaN,49.0,17885.0,2526.0,NaN,4693,240.360000,406.0,328.0,963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.333333,NaN,NaN,2.000000,3.000000,2.000000,1.000000,0.297297,0.918919,0.081081,0.324324,0.945946,0.000000,0.054054,1,0,19.653060,0.262399,51003.0,1841.471000,900.529999,7.0,21729.0,10.0,33.6,82.732750,0.000414,0.000368,0.000736,0.000000,0.000046,0.000046,0.000046,0.000000,0.032905,0.032905,0.004970,0.004970,0.956188,0.956188,0.094804,1
98313,510012.0,0.1675,NaN,NaN,94.0,34310.0,3461.0,734.0,6656,477.840000,1055.0,434.0,2161,87,81,74,68,72,67,79,91,78,300 or more,29,3.000000,2.000000,2.000000,2.000000,2.000000,1.33333

In [221]:
dataset_open['prop_gen_int'].value_counts(dropna=0).sort_index()

0.000000    9906
0.000014      39
0.000014      16
0.000016      16
0.000017      16
            ... 
0.001688     345
0.002442      55
0.002965      49
0.003717      61
inf         5665
Name: prop_gen_int, Length: 1963, dtype: int64

In [296]:
def data_process(dataset_close,a):
    
    dataset_close['prop_discharges'].fillna(dataset_close['prop_discharges'].mean(),inplace=True)
    dataset_close['hsanum'].fillna(dataset_close['hsanum'].mean(),inplace=True)
    dataset_close['beds'].fillna(dataset_close['beds'].mean(),inplace=True)
    dataset_close['bed_days'].fillna(dataset_close['bed_days'].mean(),inplace=True)
    dataset_close['dshpct'].fillna(dataset_close['dshpct'].mean(),inplace=True)
    dataset_close['gme_part_b'].fillna(dataset_close['gme_part_b'].mean(),inplace=True)
    dataset_close['gme_part_a'].fillna(dataset_close['gme_part_a'].mean(),inplace=True)
    dataset_close['medicare_days'].fillna(dataset_close['medicare_days'].mean(),inplace=True)
    dataset_close['medicaid_days'].fillna(dataset_close['medicaid_days'].mean(),inplace=True)
    dataset_close['employees'].fillna(dataset_close['employees'].mean(),inplace=True)
    dataset_close['medicare_disch'].fillna(dataset_close['medicare_disch'].mean(),inplace=True)
    dataset_close['medicaid_disch'].fillna(dataset_close['medicaid_disch'].mean(),inplace=True)
    dataset_close['doc_communicate']=dataset_close['doc_communicate'].replace({'Not Available':np.nan})
    dataset_close['doc_communicate'].fillna(dataset_close['doc_communicate'].astype(float).mean(),inplace=True)
    dataset_close['nurse_communicate']=dataset_close['nurse_communicate'].replace({'Not Available':np.nan})
    dataset_close['nurse_communicate'].fillna(dataset_close['nurse_communicate'].astype(float).mean(),inplace=True)
    dataset_close['help_quick']=dataset_close['help_quick'].replace({'Not Available':np.nan})
    dataset_close['help_quick'].fillna(dataset_close['help_quick'].astype(float).mean(),inplace=True)
    dataset_close['staff_explain']=dataset_close['staff_explain'].replace({'Not Available':np.nan})
    dataset_close['staff_explain'].fillna(dataset_close['staff_explain'].astype(float).mean(),inplace=True)
    dataset_close['pain_control']=dataset_close['pain_control'].replace({'Not Available':np.nan})
    dataset_close['pain_control'].fillna(dataset_close['pain_control'].astype(float).mean(),inplace=True)
    dataset_close['area_quiet']=dataset_close['area_quiet'].replace({'Not Available':np.nan})
    dataset_close['area_quiet'].fillna(dataset_close['area_quiet'].astype(float).mean(),inplace=True)
    dataset_close['room_clean']=dataset_close['room_clean'].replace({'Not Available':np.nan})
    dataset_close['room_clean'].fillna(dataset_close['room_clean'].astype(float).mean(),inplace=True)
    dataset_close['discharge_info']=dataset_close['discharge_info'].replace({'Not Available':np.nan})
    dataset_close['discharge_info'].fillna(dataset_close['discharge_info'].astype(float).mean(),inplace=True)
    dataset_close['recommend']=dataset_close['recommend'].replace({'Not Available':np.nan})
    dataset_close['recommend'].fillna(dataset_close['recommend'].astype(float).mean(),inplace=True)
    if a=='close':
        dataset_close['num_complete_survey']=dataset_close['num_complete_survey'].replace({'300 or more':1664.66,'300 or More':1664.66,'Between 100 and 299':200,'Not Available':np.nan,'Fewer than 100':66,'FEWER THAN 50':40})
    else:
        dataset_close['num_complete_survey']=dataset_close['num_complete_survey'].replace({'300 or more':1075.75,'300 or More':1075.75,'Between 100 and 299':191.22,'Not Available':np.nan,'Fewer than 100':73.73,'FEWER THAN 50':40})
    dataset_close['num_complete_survey'].fillna(dataset_close['num_complete_survey'].astype(float).mean(),inplace=True)
    dataset_close['survey_response_rate']=dataset_close['survey_response_rate'].replace({'Not Available':np.nan})
    dataset_close['survey_response_rate'].fillna(dataset_close['survey_response_rate'].astype(float).mean(),inplace=True)
    dataset_close['rate.x'].fillna(dataset_close['rate.x'].mean(),inplace=True)
    dataset_close['mortality'].fillna(dataset_close['mortality'].mean(),inplace=True)
    dataset_close['safety'].fillna(dataset_close['safety'].mean(),inplace=True)
    dataset_close['readmin'].fillna(dataset_close['readmin'].mean(),inplace=True)
    dataset_close['patient_exp'].fillna(dataset_close['patient_exp'].mean(),inplace=True)
    dataset_close['effective'].fillna(dataset_close['effective'].mean(),inplace=True)
    dataset_close['timely'].fillna(dataset_close['timely'].mean(),inplace=True)
    dataset_close['sex'].fillna(dataset_close['sex'].mean(),inplace=True)
    dataset_close['race.x'].fillna(dataset_close['race.x'].mean(),inplace=True)
    dataset_close['race.y'].fillna(dataset_close['race.y'].mean(),inplace=True)
    dataset_close['buyin_mo'].fillna(dataset_close['buyin_mo'].mean(),inplace=True)
    dataset_close['type_adm.x'].fillna(dataset_close['type_adm.x'].mean(),inplace=True)
    dataset_close['type_adm.y'].fillna(dataset_close['type_adm.y'].mean(),inplace=True)
    dataset_close['type_adm'].fillna(dataset_close['type_adm'].mean(),inplace=True)
    dataset_close['rate.y'].fillna(dataset_close['rate.y'].mean(),inplace=True)
    dataset_close['rural_urban'].fillna(1.0,inplace=True)
    dataset_close['pop'].fillna(dataset_close['pop'].mean(),inplace=True)
    dataset_close['unemp'].fillna(dataset_close['unemp'].mean(),inplace=True)
    dataset_close['poverty'].fillna(dataset_close['poverty'].mean(),inplace=True)
    dataset_close['under_65'].fillna(dataset_close['under_65'].mean(),inplace=True)
    dataset_close['prop_phy_pcp'].fillna(dataset_close['prop_phy_pcp'].mean(),inplace=True)
    dataset_close['prop_md_pcp'].fillna(dataset_close['prop_md_pcp'].mean(),inplace=True)
    dataset_close['prop_pcp'].fillna(dataset_close['prop_pcp'].mean(),inplace=True)
    dataset_close['prop_gi'].fillna(dataset_close['prop_gi'].mean(),inplace=True)
    dataset_close['prop_gen_int'].fillna(dataset_close['prop_gen_int'].mean(),inplace=True)
    dataset_close['prop_hosp'].fillna(dataset_close['prop_hosp'].mean(),inplace=True)
    dataset_close['prop_hosp_short'].fillna(dataset_close['prop_hosp_short'].mean(),inplace=True)
    dataset_close['prop_cah'].fillna(dataset_close['prop_cah'].mean(),inplace=True)
    dataset_close['prop_hosp_adm'].fillna(dataset_close['prop_hosp_adm'].mean(),inplace=True)
    dataset_close['prop_hosp_adm_short'].fillna(dataset_close['prop_hosp_adm_short'].mean(),inplace=True)
    dataset_close['prop_hosp_bed'].fillna(dataset_close['prop_hosp_bed'].mean(),inplace=True)
    dataset_close['prop_hosp_bed_short'].fillna(dataset_close['prop_hosp_bed_short'].mean(),inplace=True)
    dataset_close['prop_days_inp'].fillna(dataset_close['prop_days_inp'].mean(),inplace=True)
    dataset_close['prop_days_inp_short'].fillna(dataset_close['prop_days_inp_short'].mean(),inplace=True)
    dataset_close['prop_days_inp_med'].fillna(dataset_close['prop_days_inp_med'].mean(),inplace=True)
    return dataset_close

In [297]:
dataset_close=data_process(dataset_close,'close')
dataset_open=data_process(dataset_open,'open')

C:\Users\zys\Anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
<ipython-input-296-a6f5465b8007>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_close['doc_communicate']=dataset_close['doc_communicate'].replace({'Not Available':np.nan})
<ipython-input-296-a6f5465b8007>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

In [316]:
dataset_=pd.concat([dataset_open,dataset_close])
dataset_.drop(columns='provider',inplace=True)
for col in dataset_:
    dataset_[col]=pd.to_numeric(dataset_[col])
dataset_=dataset_.to_numpy()

In [318]:
train, test = train_test_split(dataset_, test_size=0.3)
X_train = train[:,:-1]
Y_train = train[:,-1].astype(int)
X_test  = test[:,:-1]
Y_test  = test[:,-1].astype(int)
X_train.shape, Y_train.shape,X_test.shape, Y_test.shape

((68388, 64), (68388,), (29310, 64), (29310,))

In [319]:
smote = SMOTE()
x_smote , y_smote = smote.fit_sample(X_train, Y_train)
x_train1, y_train1 = x_smote , y_smote
x_test1 =X_test

In [320]:
m=RandomForestClassifier()
m.fit(x_train1, y_train1)
predict_y = m.predict(x_test1)
print('ROCAUC score:',roc_auc_score(Y_test, predict_y))
print('Accuracy score:',accuracy_score(Y_test, predict_y))
print('F1 score:',f1_score(Y_test, predict_y))
print(classification_report(Y_test, predict_y))
confusion_matrix(Y_test, predict_y)

ROCAUC score: 0.8918747896891281
Accuracy score: 0.9994199931763903
F1 score: 0.8721804511278195
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     29236
           1       0.98      0.78      0.87        74

    accuracy                           1.00     29310
   macro avg       0.99      0.89      0.94     29310
weighted avg       1.00      1.00      1.00     29310



array([[29235,     1],
       [   16,    58]], dtype=int64)

In [323]:
X_train, X_test, y_train, y_test = train_test_split(dataset_[:,:-1],
                                                    dataset_[:,-1],
                                                    test_size=0.3,
                                                    stratify=dataset_[:,-1]
                                                    )

In [302]:
pipeline = imbpipeline(steps = [['smote', SMOTE()],
                                
                                ['classifier', RandomForestClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True
                                       )
    
param_grid = {'classifier__n_estimators':[100,200]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_RF_s = grid_search.best_score_
test_score_RF_s = grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_RF_s}\nTest score: {test_score_RF_s}')

Cross-validation score: 0.8885653949685646
Test score: 0.8095238095238095


In [303]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                
                                ['classifier', RandomForestClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True
                                       )
    
param_grid = {'classifier__n_estimators':[100,200]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_RF_a = grid_search.best_score_
test_score_RF_a = grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_RF_a}\nTest score: {test_score_RF_a}')

Cross-validation score: 0.7858049986728993
Test score: 0.787878787878788


In [304]:
pipeline = imbpipeline(steps = [['SVMSMOTE', SVMSMOTE()],
                                
                                ['classifier', RandomForestClassifier()]])

stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True
                                       )
    
param_grid = {'classifier__n_estimators':[100,200]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_RF_svm = grid_search.best_score_
test_score_RF_svm = grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_RF_svm}\nTest score: {test_score_RF_svm}')

Cross-validation score: 0.8737236336452439
Test score: 0.7967479674796748


In [305]:
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                
                                ['classifier', LogisticRegression()]])
    
param_grid = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_LR_s = grid_search.best_score_
test_score_LR_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_LR_s}\nTest score: {test_score_LR_s}')

Cross-validation score: 0.009821982549619641
Test score: 0.008883248730964468


C:\Users\zys\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [306]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                
                                ['classifier', LogisticRegression()]])
    
param_grid = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_LR_a = grid_search.best_score_
test_score_LR_a= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_LR_a}\nTest score: {test_score_LR_a}')

Cross-validation score: 0.009451351350857009
Test score: 0.011097239561659035


C:\Users\zys\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [328]:
pipeline = imbpipeline(steps = [['SVMSMOTE', SVMSMOTE()],
                                
                                ['classifier', LogisticRegression()]])
    
param_grid = {'classifier__C':[0.001, 0.01, 0.1, 1, 10, 100, 1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_LR_svm = grid_search.best_score_
test_score_LR_svm= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_LR_svm}\nTest score: {test_score_LR_svm}')

Cross-validation score: 0.008703604952573343
Test score: 0.007682249101013403


C:\Users\zys\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [310]:
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                
                                ['classifier', DecisionTreeClassifier()]])
    
param_grid = {'classifier__criterion':['gini','entropy'],'classifier__max_depth':[5,200,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_DT_s = grid_search.best_score_
test_score_DT_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_DT_s}\nTest score: {test_score_DT_s}')

Cross-validation score: 0.8057644856632349
Test score: 0.8258064516129032


In [311]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                
                                ['classifier', DecisionTreeClassifier()]])
    
param_grid = {'classifier__criterion':['gini','entropy'],'classifier__max_depth':[5,200,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_DT_a = grid_search.best_score_
test_score_DT_a= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_DT_a}\nTest score: {test_score_DT_a}')

Cross-validation score: 0.8240073027383769
Test score: 0.7804878048780488


In [312]:
pipeline = imbpipeline(steps = [['SVMSMOTE',SVMSMOTE()],
                                
                                ['classifier', DecisionTreeClassifier()]])
    
param_grid = {'classifier__criterion':['gini','entropy'],'classifier__max_depth':[5,200,500]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_DT_svm = grid_search.best_score_
test_score_DT_svm= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_DT_svm}\nTest score: {test_score_DT_svm}')

Cross-validation score: 0.8656249988639193
Test score: 0.8219178082191781


In [313]:
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                
                                ['classifier', KNeighborsClassifier()]])
    
param_grid = {'classifier__n_neighbors':[5,1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_knn_s = grid_search.best_score_
test_score_knn_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_knn_s}\nTest score: {test_score_knn_s}')

Cross-validation score: 0.21264340344285024
Test score: 0.20399113082039913


In [314]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                
                                ['classifier', KNeighborsClassifier()]])
    
param_grid = {'classifier__n_neighbors':[5,1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_knn_a = grid_search.best_score_
test_score_knn_a= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_knn_a}\nTest score: {test_score_knn_a}')

Cross-validation score: 0.20113496103119025
Test score: 0.1987041036717063


In [329]:
pipeline = imbpipeline(steps = [['SVMSMOTE', SVMSMOTE()],
                                
                                ['classifier', KNeighborsClassifier()]])
    
param_grid = {'classifier__n_neighbors':[5,1000]}
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_knn_svm = grid_search.best_score_
test_score_knn_svm= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_knn_svm}\nTest score: {test_score_knn_svm}')

C:\Users\zys\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [nan nan]
  warnings.warn(


Cross-validation score: nan
Test score: 0.3


In [324]:
pipeline = imbpipeline(steps = [['SMOTE', SMOTE()],
                                
                                ['classifier', XGBClassifier()]])
    
param_grid = {'classifier__min_child_weight': [1, 5, 10]
        }
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_s = grid_search.best_score_
test_score_xgb_s= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_s}\nTest score: {test_score_xgb_s}')

C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:54:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross-validation score: 0.9150460589304178
Test score: 0.9395973154362416


In [325]:
pipeline = imbpipeline(steps = [['ADASYN', ADASYN()],
                                
                                ['classifier', XGBClassifier()]])

grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_a = grid_search.best_score_
test_score_xgb_a= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_a}\nTest score: {test_score_xgb_a}')

C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:55:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross-validation score: 0.9193449647657636
Test score: 0.9315068493150684


In [326]:
pipeline = imbpipeline(steps = [['SVMSMOTE', SVMSMOTE()],
                                
                                ['classifier', XGBClassifier()]])
    
grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=stratified_kfold,
                           n_jobs=-1)

grid_search.fit(X_train, y_train)
cv_score_xgb_svm = grid_search.best_score_
test_score_xgb_svm= grid_search.score(X_test, y_test)
print(f'Cross-validation score: {cv_score_xgb_svm}\nTest score: {test_score_xgb_svm}')

C:\Users\zys\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[06:57:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Cross-validation score: 0.9402840971863119
Test score: 0.8961038961038962


In [332]:
models = pd.DataFrame({
    'Model': ['Random Forest', 'Logistic Regression', 
              'Decision Tree','KNN',  
              'Gradient Boosting'],
    'SMOTE': [cv_score_RF_s,cv_score_LR_s,cv_score_DT_s,cv_score_knn_s,cv_score_xgb_s],
    'ADASYN': [cv_score_RF_a,cv_score_LR_a,cv_score_DT_a,cv_score_knn_a,cv_score_xgb_a],
    'SVMSMOTE':[cv_score_RF_svm,cv_score_LR_svm,cv_score_DT_svm,cv_score_knn_svm,cv_score_xgb_svm]})

In [333]:
models

,Model,SMOTE,ADASYN,SVMSMOTE
0,Random Forest,0.888565,0.785805,0.873724
1,Logistic Regression,0.009822,0.009451,0.008704
2,Decision Tree,0.805764,0.824007,0.865625
3,KNN,0.212643,0.201135,NaN
4,Gradient Boosting,0.915046,0.919345,0.940284
